In [1]:
import pandas as pd
import re
from random import randint
import spacy
from time import time
from pathlib import Path
from pathlib import *

# Folketingstidende datawrangling

Det følgende script er en renskrivning af et "trial and error" script jeg tidligere har udarbejdet. Dette script kan findes på min github:

### Formål
Dette script ønsker at formatere "folketingstidende" på en sådan måde at der kan laves en digitalhistorisk analyse af data. Formålet med scriptet er da at "genkende" når en ny person begynder at tale fra folketingsets talerstol, så folektingstidende kan opdeles i folketingstaler og udtræke metadateen "hvem der taler" samt "hvilket parti tilhører taleren". Derudover skal scriptet søge at genkende når der er "fejl" i data som skal sorteres fra. Her er primært tale om at sortere marginer fra. Da der er tale om en "OCR" læsning af folketingstidende starter hver side med marginen på siden, som hvis denne ikke sorteres fra, vil indgå som en del af et folketingsmedlem eller ministers tale. 

### Genkendelse af talere:

I det følgende vil jeg søge at benytte mig af regex til at genkende når en ny person indtager folketingsets talerstol. Til dette vil jeg udnytte hvordan folketingstidende er "formateret" af udgiver. Når en ny person indtager folketingets talerstol vil dette blive skrevet som "Navn på talende folketingsmedlem (Parti som folketingsmedlem tilhører)". Det er altså dette jeg vil søge at genkende.

Jeg starter med at indlæse en del af folketingstidende som prøve.

In [2]:
df = pd.read_csv('../../Folketingstidende/Hele/19901.csv',sep=';', encoding = "latin1")

Jeg finder ny søjlenavnene:

In [3]:
df.columns.values.tolist()

['Unnamed: 0', 'spalte', 'TextBlock', 'TopMargin', 'indhold', 'år', 'samling']

Det er søjlen "indhold" der primært har interesse.

In [4]:
FT_data = [indhold for indhold in df["indhold"]]

In [5]:
FT_data[0:10]

['(1) l 2/10 90: Folketingets åbning 2',
 '1. møde',
 'Tirsdag den 2. oktober 1990 kl. 12.00',
 'Aldersformanden (Ninn-Hansen):',
 'På denne første tirsdag i oktober er Folketinget trådt sammen for at indlede folketingsåret 1990-91, og det sker som bekendt i henhold til bestemmelser i grundloven herom.',
 'Som den, der længst har været medlem af Tinget, er det min opgave og pligt at lede valget af præsidium og af tingsekretærer.',
 'Til formand har et flertal bestående af Det Konservative Folkeparti, Venstre, Fremskridtspartiet, Det Radikale Venstre, Centrum-Demokraterne og Kristeligt Folkeparti indstillet H. P. Clausen (KF).',
 'Over for mig foreligger der ikke andre indstillinger. Hvis ingen begærer afstemning, vil jeg betragte den her foretagne indstilling som vedtaget. (Ophold). Der er ingen, der begærer afstemning, og dermed er H. P. Clausen valgt.',
 'Til næstformænd har Tingets fire største partier - bortset fra det parti, som formanden tilhører - udpeget følgende medlemmer:',
 

Her viser jeg de første 10 OCR-læste afsnit i folketingstidende.

Jeg vil nu søge at genkende når der er folk der taler fra folketingsets talerstol.

In [6]:
Prøve_FT = []
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)", "(KRF)"] #liste af partier
for sentence in FT_data:
    #if len(re.findall("(\w{1,10}\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) > 0:
    if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0:
        if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
            if len(re.findall("Ny stedf.", sentence)) > 0:
                pass
            elif len(re.findall("Udtræder:", sentence)) > 0:
                pass
            elif len(re.findall("Nyt medl", sentence)) > 0:
                pass
            elif len(re.findall(" af", sentence.lower())) > 0:
                pass
            elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                Prøve_FT.append(sentence)

Jeg kan nu undersøges hvad der genkendes:

In [7]:
Prøve_FT.sort(key = len, reverse = True)

In [8]:
len(Prøve_FT)

2624

In [9]:
Prøve_FT[0:100]

['forkastedes, idet 13 (SF) stemte for, 56 (KF, V, FP, RV og CD) imod; 34 (S og KRF) stemte hverken for eller imod.',
 've Niemann (FP) har meddelt mig, at de til justitsministeren ønsker at stille følgende:',
 'Af Birgitte Husmark (SF) m.fl. (Forespørgslen anmeldt 4/10 90).',
 'Af Kirsten Jacobsen (FP) m.fl. (Forespørgslen anmeldt 8/11 90).',
 'Af Margrete Auken (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 'Af Tommy Dinesen (SF) m.fl. (Forespørgslen anmeldt 10/10 90).',
 'Af Pia Kjærsgaard (FP) m.fl. (Forespørgslen anmeldt 8/11 90).',
 'Af Aage Frandsen (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 'Af Inger Stilling Pedersen (KRF) m.fl. (Fremsat 8/11 90).',
 'Af Lysholm Christensen (KRF) m.fl. (Fremsat 31/10 90).',
 '15 (SF) stemte hverken for eller imod.',
 'Af Rahbæk Møller (SF), Dorte Bennedsen',
 'Af Rahbæk Møller (SF), Dorte Bennedsen',
 'Af Inger Stilling Pedersen (KRF) m.fl.',
 'Nyt med},: Else Winther Andersen (V)',
 '16-17 Hanne Thanning Jacobsen (SF)',
 '16-17 Hanne Than

Det ovenstående søger at genkende når der er et menigt medlem(hermed ment ikke formanden eller en minister) som ønsker at tale. Problemet er her at opremsningen af medlemmer af folketinget i starten af folketingsåret genkendes som "en taler" samt at metatale lavet af formanden genkendes eks. "Medlem af folketinget Askjær Jørgensen (FP) har meddelt mig, at han ønsker til industriministeren at stille følgende:". I begge tilfælde ser jeg det dog som mindre problematisk. 
I første tilfælde vil de "genkendte" talere blive tildelt en "tale" som er tom, altså at disse ikke siger noget, hvilket vil blive sorteret fra når jeg udtrækker de folketingstaler jeg ønsker at analysere. 
I andet tilfælde af problemet er ligeledes mindre problematisk. Her er der tale om at folketingets formand fremsætter et menigt medlems holdninger, og deraf ser jeg det ikke problematisk at "karakterisere" disse som folketingstaler.

### Genkendelse af ministre:

In [10]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [11]:
ministre_FT.sort(key = len, reverse = True)

In [12]:
ministre_FT[0:1000]

['Industriministeren (Anne Birgitte Lundholt) har meddelt mig, at hun ønsker skriftligt at fremsætte:',
 'Industriministeren (Anne Birgitte Lundholt) har meddelt mig, at hun ønsker skriftligt at fremsætte:',
 'Industriministeren (Anne Birgitte Lundholt) har meddelt mig, at hun ønsker skriftligt at fremsætte:',
 'Kirkeministeren (Torben Rechendorff) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte :',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte :',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Finansministeren (Henning Dyremose) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Indenrigsministeren (Thor Pedersen) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Trafikministeren (Knud Østergaard) har meddelt mig, at han ønsker skriftligt at fremsætte :',
 'Indenrigsmi

In [13]:
ministre_FT = []
for sentence in FT_data:
    if len(sentence) < 100:
        if len(re.findall("\w*minister[\w ]*\([\w-]*\)", sentence.lower())) > 0:
            ministre_FT.append(sentence)

In [14]:
ministre_FT

['Justitsministeren (Engell) har meddelt mig, at han ønsker skriftligt at fremsætte:',
 'Justitsministeren (Engell):',
 'Landbrugsministeren (Tørnæs) har meddelt mig, at han ønsker skriftligt at fremætte:',
 'Landbrugsministeren (Tørnæs):',
 'Af justitsministeren (Engell).',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Udenrigsministeren (Ellemann-Jensen):',
 'Af justitsministeren (Engell).',
 'Justitsministeren (Engell):',
 'Forslag til lov om frø, kartofler og planter. Af landbrugsministeren (Tørnæs).',
 'Justitsministeren (Engell) har meddelt mig, at han ønsker skriftligt af fremsætte:',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',
 'Justitsministeren (Engell):',

Det ovenstående genkender meget godt når der er tale om en minister.

In [15]:
p = "Energiministeren (Bilgrav-Nielsen)"
len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", p.lower()))

1

### Genkendelse af Formanden:

In [16]:
count_formanden = 0
for sentence in FT_data:
    if sentence.lower() == "formanden" or sentence.lower() == "formanden:":
        count_formanden += 1
count_formanden

193

### Genkendelse af næstformand

In [17]:
n = 0
for sentence in FT_data:
    if re.search("næstformand \(\w*\):$", sentence.lower()) or re.search("næstformand \(\w* \w*\):$", sentence.lower()):
        n += 1
        
n

123

Jeg undersøger nu om der kan være nogle datapunkter der er blevet misset:

In [18]:
formanden_problem = []
for sentence in FT_data:
    if len(sentence) <= 15:
        if len(re.findall("formanden", sentence.lower())) > 0:
            if sentence.lower() != "formanden" and sentence.lower() != "formanden:":
                formanden_problem.append(sentence)
formanden_problem[0:10]

['[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]',
 '[Formanden]']

I det ovenstående er der tale om at der står i margenen af folketingstidende, at det er formanden der taler, det er derfor ikke problematisk at disse sorteres fra.

### Marginer sorteres fra:

In [19]:
hele_margin = []
for sentence in FT_data:
    if len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
        if len(sentence) < 200:
            hele_margin.append(sentence)
print(len(hele_margin))
hele_margin[0:10]

737


['3 2/10 90: Folketingets åbning 4',
 '5 2/10 90: Statsministerens åbningstale 6',
 '7 2/10 90; Statsministerens åbningstale 8',
 '9 2/10 90: Statsministerens åbningstale 10',
 '11 2/10 90: Statsministerens åbningstale 12',
 '13 2/10 90: Statsministerens åbningstale 14',
 '15 2/10 90: Statsministerens åbningstale 16',
 '17 2/10 90 : Statsministerens åbningstale. Skriftlig del 18',
 '19 2/10 90: Statsministerens åbningstale. Skriftlig del 20',
 '21 2/10 90: Statsministerens åbningstale. Skriftlig del 22']

In [20]:
hele_margin_prøve = []
for sentence in FT_data:
    if len(sentence) <= 170: 
        if len(re.findall("[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower()))  > 0:
            hele_margin_prøve.append(sentence)
print(len(hele_margin_prøve))
hele_margin_prøve[0:10]

952


['3 2/10 90: Folketingets åbning 4',
 '5 2/10 90: Statsministerens åbningstale 6',
 '7 2/10 90; Statsministerens åbningstale 8',
 '9 2/10 90: Statsministerens åbningstale 10',
 '11 2/10 90: Statsministerens åbningstale 12',
 '13 2/10 90: Statsministerens åbningstale 14',
 '15 2/10 90: Statsministerens åbningstale 16',
 '17 2/10 90 : Statsministerens åbningstale. Skriftlig del 18',
 '19 2/10 90: Statsministerens åbningstale. Skriftlig del 20',
 '21 2/10 90: Statsministerens åbningstale. Skriftlig del 22']

In [21]:
hele_margin_prøve.sort(key = len, reverse = True)
hele_margin_prøve[0:10]

['2255 21/11 90: 1. beh. af f. t. I. vedr. tilsyn med begraveIseskasser og ligbrændingsforeninger m.v. 2256',
 '2257 21/11 90: 1. beh. af f. 1. I. vedr. tilsyn med begravelseskasser og ligbrændingsforeninger m.v. 2258',
 '2253 21/11 90: 1. beh. af f. 1.1. vedr. Det Europæiske Økonomiske Fællesskabs forordninger m.v. 2254',
 '(125) 1985 15/11 90: 1. beh. af f. t. beslutn. om ophævelse af sanktionerne over for Sydafrika 1986',
 '(69) 1089 25/10 90: 1. beh. äff. 1.1. vedr. erhverv i Danmark for statsborgere i EF og Norden 1090',
 '1347 31/10 90: 1. beh. af f. t. beslutn. om støtte til erhvervsdrivende ved vejarbejder m.v. 1348',
 '1349 31/10 90: 1. beh. af f. t. beslutn. om støtte til erhvervsdrivende ved vejarbejder m.v. 1350',
 '1351 31/10 90: 1. beh. af f. t. beslutn. om støtte til erhvervsdrivende ved vejarbejder m.v. 1352',
 '1353 31/10 90: 1. beh. af f. t. beslutn. om støtte til erhvervsdrivende ved vejarbejder m.v. 1354',
 '1355 31/10 90: 1. beh. af f. t. beslutn. om støtte til erh

Det er klart at det ovenstående er marginerne på hver side. Hvis man dog følger tallet til venstre, som er det første spalte(af to) på hver side af folketingstidende, kan man dog se at det ikke er alle marginer der findes her.

In [22]:
dato_alene = []
for sentence in FT_data:
    if len(re.findall("[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                dato_alene.append(sentence)
print(len(dato_alene))
dato_alene[0:10]

1551


['(1) l 2/10 90: Folketingets åbning 2',
 '3 2/10 90: Folketingets åbning 4',
 '5 2/10 90: Statsministerens åbningstale 6',
 '7 2/10 90; Statsministerens åbningstale 8',
 '9 2/10 90: Statsministerens åbningstale 10',
 '11 2/10 90: Statsministerens åbningstale 12',
 '13 2/10 90: Statsministerens åbningstale 14',
 '15 2/10 90: Statsministerens åbningstale 16',
 '17 2/10 90 : Statsministerens åbningstale. Skriftlig del 18',
 '19 2/10 90: Statsministerens åbningstale. Skriftlig del 20']

Her kan det også ses at der er tale om marginer. Disse skal derfor ligeledes sorteres fra. Problemet er her at spaltenumrene da er sorteret som individuelle datapunkter. Jeg prøver da også at sortere disse fra.

In [23]:
tal_genkendt = []
for sentence in FT_data:
    if re.fullmatch("[1-9]{1,10}", sentence.lower()): #tilføjet til at fange headers på sider
        tal_genkendt.append(sentence)
tal_genkendt[0:10]

['534', '919', '1125', '1126', '1127', '1128', '1134', '1136', '1142', '1143']

Hvis man kigger tallene igennem er det meget plausibelt at der her er tale om de manglende spaltenumre. Disse skal altså også sorteres fra.

I marginerne optræder altså også en navn på vedkommende der er den "tallende" når der skiftes spalte. Disse vil jeg også forsøge at sortere fra, da jeg ikke ønsker at en taler fra folketingets talerstols eget navn skal indgå i dennes tale. Heldigvis har disse navne også en relativt nem formatering, nemlig \[navn\]. Jeg søger altså efter datapunker der matcher denne formatering.

In [24]:
tallere_fra_forrige_side = []
for sentence in FT_data:
    if re.fullmatch("\[.{1,100}\]", sentence.lower()):
        tallere_fra_forrige_side.append(sentence)
print(len(tallere_fra_forrige_side))

1027


In [25]:
tallere_fra_forrige_side[0:10]

['[Formanden]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]',
 '[Statsministeren]']

### Tallere fra forrige side som ikke er blevet læst som seperate enheder:

In [26]:
tallere_fra_forrige_side_læst_forkert = []
for sentence in FT_data:
    if not re.fullmatch("\[.{1,100}\]", sentence.lower()):
        if not re.fullmatch("\[.{1,100}\].", sentence.lower()):
            if re.match("^\[.{1,100}\]", sentence.lower()):
                tallere_fra_forrige_side_læst_forkert.append(sentence)
print(len(tallere_fra_forrige_side_læst_forkert))

1


In [27]:
tallere_fra_forrige_side_læst_forkert

["[Erling Christensen] '"]

Her findes de forskellige navne fra marginerne, som skal sorteres fra.

## Sammensætning af det ovenstående

I det følgende vil jeg sætte alt det ovenstående sammen, med det formål at udtrække folketingstaler, samt hvem der taler, og sortere marginer fra.

In [28]:
indhold = FT_data
talere = []
tale = []
parti = []
count_folketing = 0
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif re.search("næstformand \(\w*\)", sentence.lower()) or re.search("næstformand \(\w* \w*\)", sentence.lower()):
            talere.append(sentence.lower())
            parti.append("næstformanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\([\w ]*\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence

Jeg laver nu en funktion ud af dette:

In [29]:
def FT_taler(indhold):
    talere = []
    tale = []
    parti = []
    count_folketing = 0
    Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)", "(KrF)"]
    for index in range(len(indhold)):
        sentence = indhold[index]
        if len(re.findall("(^.{1,50} \([A-Z]+\)|.{1,50} \(KrF\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                if len(re.findall("Ny stedf.", sentence)) > 0:
                    pass
                elif len(re.findall("Udtræder:", sentence)) > 0:
                    pass
                elif len(re.findall("Nyt medl", sentence)) > 0:
                    pass
                elif len(re.findall(" af", sentence.lower())) > 0:
                    pass
                elif re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0] in Partier_i_folketinget:
                    talere.append(sentence)
                    parti.append(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)[0])
                    tale.append("")
                    count_folketing += 1
                else:
                    if len(tale) >= 1:
                        tale[-1] = tale[-1] + " " + sentence
            else: #ellers er det en tale
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
                #if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
        elif sentence.lower() == "formanden" or sentence.lower() == "formanden:": #tjekker om det er formanden der taler
            talere.append("formanden")
            parti.append("Formanden")
            tale.append("")
        elif re.search("næstformand \(\w*\):$", sentence.lower()) or re.search("næstformand \(\w* \w*\):$", sentence.lower()):
            talere.append(sentence.lower())
            parti.append("næstformanden")
            tale.append("")
        elif len(sentence) < 100: #Tjekker om det er en minister der taler
            if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\)", sentence.lower())) > 0:
                talere.append(sentence)
                parti.append("Regeringen")
                tale.append("")
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif len(re.findall("^[1-9]{1,5} [1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
        elif len(re.findall("^[1-9]{1,2}/[1-9]{1,2}", sentence.lower())) > 0: #tilføjet til at fange headers på sider
            if len(sentence) < 200:
                pass
            else:
                if len(tale) >= 1:
                    tale[-1] = tale[-1] + " " + sentence
        elif re.fullmatch("\[.{1,100}\]", sentence.lower()):
            pass
        elif re.fullmatch("[1-9]{1,10}", sentence.lower()):
            pass
        else:
            if len(tale) >= 1:
                tale[-1] = tale[-1] + " " + sentence
    return tale, talere, parti

Ovenstående funktion danner basis for formatering af data til senere analyse.

### Formatering af 90'ernes data:

Ved en senere analyse kunne det tyde på at 90'ernes data er dårligere opdelt end det tidligere, jeg undersøger:

In [30]:
path = "../../Folketingstidende/Hele/"
dir_path = Path(f"{path}")
csv_files = list(dir_path.glob("*.csv"))

In [31]:
csv_files.sort()
csv_files[0:5]

[PosixPath('../../Folketingstidende/Hele/19531.csv'),
 PosixPath('../../Folketingstidende/Hele/19541.csv'),
 PosixPath('../../Folketingstidende/Hele/19551.csv'),
 PosixPath('../../Folketingstidende/Hele/19561.csv'),
 PosixPath('../../Folketingstidende/Hele/19562.csv')]

In [32]:
csv_files[49:61]

[PosixPath('../../Folketingstidende/Hele/19901.csv'),
 PosixPath('../../Folketingstidende/Hele/19902.csv'),
 PosixPath('../../Folketingstidende/Hele/19911.csv'),
 PosixPath('../../Folketingstidende/Hele/19921.csv'),
 PosixPath('../../Folketingstidende/Hele/19931.csv'),
 PosixPath('../../Folketingstidende/Hele/19941.csv'),
 PosixPath('../../Folketingstidende/Hele/19951.csv'),
 PosixPath('../../Folketingstidende/Hele/19961.csv'),
 PosixPath('../../Folketingstidende/Hele/19971.csv'),
 PosixPath('../../Folketingstidende/Hele/19972.csv'),
 PosixPath('../../Folketingstidende/Hele/19981.csv'),
 PosixPath('../../Folketingstidende/Hele/19991.csv')]

In [33]:
Folketingstaler_i_90erne = {}
for fil in csv_files[49:61]:
    if str(fil)[-9:-5] in Folketingstaler_i_90erne:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        for tale in taler:
            Folketingstaler_i_90erne[str(fil)[-9:-5]].append(tale)
    else:
        folketingsår = pd.read_csv(fil ,sep=';', encoding = "latin1")
        taler = folketingsår["indhold"]
        Folketingstaler_i_90erne[str(fil)[-9:-5]] = []
        for tale in taler:
            Folketingstaler_i_90erne[str(fil)[-9:-5]].append(tale)

### Undersøgelse af ministre 1990'erne

In [34]:
ministre_FT = []
for år in Folketingstaler_i_90erne:
    FT_data = Folketingstaler_i_90erne[år]
    for sentence in FT_data:
        if len(sentence) >= 100:
            if len(re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\):", sentence.lower())) > 0:
                tale = re.findall("\w*minister[\w ]*\(([\w-]*|[\w ]*)\):", sentence.lower())
                ministre_FT.append(sentence)

In [35]:
ministre_FT

['Socialministeren (Else Winther Andersen): Efter § 23 i lov om social pension træffer kommunalbestyrelsen afgørelse om pension efter § 14, stk. 3, og videresender ansøgninger om pension efter § 14, stk. 1 og 2, og §§ 16 og 18 med indstilling til revaliderings- og pen- sionsnævnet.',
 'Vi skal selvfølgelig have nogle oplagte emner at stille spørgsmål om i udvalget, men ministeren nævnte vist, at der var 20.000 sommerhuse i Danmark. (Miljøministeren (Svend Auken): 200.000!). Ja, det kan vist bedre passe, for i Berlingske Tidende den 14. marts oplyser tre politimestre i Nordsjælland nemlig, at alene inden for deres område har de 30.000 sommerhuse, de skal kontrollere. Undskyld, så hørte jeg altså forkert.',
 'Statsministeren (Poul Nyrup Rasmussen): Regeringen fremlagde ved sin tiltræden i sidste uge regeringsgrundlaget »Fælles Fremtid«. Her redegjorde vi for hovedlinjerne i vores politik de næste fire år. Det overordnede mål er at skabe et bedre samfund for flere, et samfund med færre og

Problem og forsøges fikset:

### Menige folketingsmedlemmer:

In [36]:
Prøve_FT = []
længder_lists = []
Partier_i_folketinget = ["(A)", "(SF)", "(S)", "(RV)", "(KF)", "(V)", "(FP)", "(CD)", "(VS)",
                             "(KrF)", "(DR)", "(UP)", "(Grønl.)", "(Fær.)", "(DF)", "(EL)"] #liste af partier
for år in Folketingstaler_i_90erne:
    FT_data = Folketingstaler_i_90erne[år]
    for sentence in FT_data:
        if len(re.findall("(^.{1,50} \([A-Z]+\)|^.{1,50} \(KrF\)|^.{1,50} \(Grønl.\)|^.{1,50} \(Fær.\))", sentence)) > 0: #Tjekker om det er et menigt medlem
            if len(re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)) < 2:
                if len(re.findall("(\([A-Z]+\)|\(KrF\))", sentence)) < 2:
                    if len(re.findall("Ny stedf.", sentence)) > 0:
                        pass
                    elif len(re.findall("Udtræder:", sentence)) > 0:
                        pass
                    elif len(re.findall("Nyt medl", sentence)) > 0:
                        pass
                    elif len(re.findall(" af", sentence.lower())) > 0:
                        pass
                    elif re.findall("(\([A-Z]+\)|\(KrF\)|\(Grønl.\)|\(Fær.\))", sentence)[0] in Partier_i_folketinget:
                        if len(sentence) > 35:
                            længder_lists.append(re.split("(^.{1,50} \([A-Z]+\)|^.{1,50} \(KrF\)|^.{1,50} \(Grønl.\)|^.{1,50} \(Fær.\))", sentence))
                        Prøve_FT.append(sentence)

In [37]:
Prøve_FT.sort(key = len, reverse = True)

In [38]:
længder_lists

[['', 'Af Margrete Auken (SF)', ' m.fl. (Forespørgslen anmeldt 3/10 90).'],
 ['', 'Af Aage Frandsen (SF)', ' m.fl. (Forespørgslen anmeldt 3/10 90).'],
 ['', 'Af Birgitte Husmark (SF)', ' m.fl. (Forespørgslen anmeldt 4/10 90).'],
 ['', 'Af Tommy Dinesen (SF)', ' m.fl. (Forespørgslen anmeldt 10/10 90).'],
 ['', '15 (SF)', ' stemte hverken for eller imod.'],
 ['',
  'forkastedes, idet 13 (SF)',
  ' stemte for, 56 (KF, V, FP, RV og CD) imod; 34 (S og KRF) stemte hverken for eller imod.'],
 ['',
  've Niemann (FP)',
  ' har meddelt mig, at de til justitsministeren ønsker at stille følgende:'],
 ['', 'Af Rahbæk Møller (SF)', ', Dorte Bennedsen'],
 ['', 'Af Rahbæk Møller (SF)', ', Dorte Bennedsen'],
 ['', 'Af Kirsten Jacobsen (FP)', ' m.fl. (Forespørgslen anmeldt 8/11 90).'],
 ['', 'Af Pia Kjærsgaard (FP)', ' m.fl. (Forespørgslen anmeldt 8/11 90).'],
 ['', 'Nyt med},: Else Winther Andersen (V)', ''],
 ['', 'VI. Bornholms Amtskreds: Eva Fatum (S)', ','],
 ['', 'Af Kirsten Jacobsen (FP)', ' m.f

In [39]:
len(Prøve_FT)

135214

In [40]:
Prøve_FT

['Bent Hindrup Andersen (EL): Århusproblemet, som har medført dette lovforslag, har man selv stoppet i Århus, så vidt jeg kan se. Jeg tror, alle ordførerne har set den telefax, man har sendt ud, og vi synes egentlig, det er en bedre måde at løse problemet på at fastholde refusionen på 48 pet. Det undrer mig i øvrigt, hvordan ministeren umiddelbart kan ned- sætte den til 40 pet., når der er handlet ud fra et regelsæt, hvor man har fået en udmelding om, at den var på 48 pct.. Så kan man jo sige, at de penge, der trods alt er givet til Århus, ikke er givet dårligt ud, for det er jo nogle teateraktiviteter, som jeg tror er helt i orden.',
 'Kristian Thulesen Dahl (DF): Indenrigsministeren har flere gange henvist til, at det er et andet ressortministerium, og dertil skal jeg bare sige, at da det var den tidligere indenrigsminister, Birte Weiss, der havde ansvaret for Indenrigsministeriet, var hun også draget ind i disse sager, var i samråd om dem og var slet ikke uvillig til at gå ind i dis

In [41]:
re.split("(^.{1,50} \([A-Z]+\):|.{1,50} \(KrF\):)", Prøve_FT[0])

['',
 'Bent Hindrup Andersen (EL):',
 ' Århusproblemet, som har medført dette lovforslag, har man selv stoppet i Århus, så vidt jeg kan se. Jeg tror, alle ordførerne har set den telefax, man har sendt ud, og vi synes egentlig, det er en bedre måde at løse problemet på at fastholde refusionen på 48 pet. Det undrer mig i øvrigt, hvordan ministeren umiddelbart kan ned- sætte den til 40 pet., når der er handlet ud fra et regelsæt, hvor man har fået en udmelding om, at den var på 48 pct.. Så kan man jo sige, at de penge, der trods alt er givet til Århus, ikke er givet dårligt ud, for det er jo nogle teateraktiviteter, som jeg tror er helt i orden.']

In [42]:
p = 'Af Kirsten Jacobsen (FP) m.fl. (Forespørgslen anmeldt 8/10 92).'

In [43]:
print(re.match("^Af ", p))

<re.Match object; span=(0, 3), match='Af '>


In [44]:
type(p)

str

In [45]:
af = []
for år in Folketingstaler_i_90erne:
    FT_data = Folketingstaler_i_90erne[år]
    for sentence in FT_data:
        if len(re.findall("(^.{1,50} \([A-Z]+\)|^.{1,50} \(KrF\)|^.{1,50} \(Grønl.\)|^.{1,50} \(Fær.\))", sentence)) > 0 and re.match("^Af ", sentence):
            af.append(sentence)

In [46]:
af

['Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Stavad (S) m.fl.',
 'Af Stavad (S) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Jens Thoft (SF) m.fl.',
 'Af Stavad (S) m.fl.',
 'Af Stavad (S) m.fl.',
 'Af Margrete Auken (SF) m.fl.',
 'Af Aage Frandsen (SF) m.fl.',
 'Af Duetoft (CD) m.fl.',
 'Af Margrete Auken (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 'Af Aage Frandsen (SF) m.fl. (Forespørgslen anmeldt 3/10 90).',
 'Af Duetoft (CD) m.fl.',
 'Af Birgitte Husmark (SF) m.fl.',
 'Af Pelle Voigt (SF) m.fl.',
 'Af Ivar Nørgaard (S), Grove (KF), Gade (SF), Bjørn Elmquist (V), Annette Just (FP), Larsen-Ledet (RV), Duetoft (CD) og Kofod- Svendsen (KRF).',
 'Af Birgitte Husmark (SF) m.fl. (Forespørgslen anmeldt 4/10 90).',
 'Af Pelle Voigt (SF) m.fl.',
 'Af Ivar Nørgaard (S), Grove (KF), Gade (SF), Bjørn Elmq

## Undersøgelse af 2000 data: